In [21]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carregando e preparando o dataset Iris com duas classes
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([0, 1])]  # Mantém apenas as classes 0 e 1

# Separando os dados de treino e teste
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Regressão Logística
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade da classe prevista
    predicted_class = clf.predict(instance.reshape(1, -1))[0]
    prob_predicted_class = clf.predict_proba(instance.reshape(1, -1))[0][predicted_class]

    # Calcula a probabilidade da outra classe
    prob_other_class = clf.predict_proba(instance.reshape(1, -1))[0][1 - predicted_class]

    # Calcula a diferença de probabilidade para cada atributo
    delta_features = [(feature,
                       abs(clf.coef_[0][i] * instance[i]))
                      for i, feature in enumerate(features)]

    # Ordena os atributos pela diferença de probabilidade (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = prob_predicted_class - prob_other_class
    current_threshold = 0
    for idx, (feature, delta) in enumerate(delta_features): 
        current_threshold += delta
        pi_explanation.append((feature, instance[features.index(feature)]))
        if current_threshold > threshold:
            break

    return pi_explanation

# Selecionando a primeira instância do conjunto de teste como um array NumPy
instance = X_test.iloc[0].to_numpy()

# Calculando a PI-explicação para a instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, list(X.columns))

# Exibindo a PI-explicação
print(f"PI-explicação Log-Linear para a instância {instance}: {pi_explanation}")


PI-explicação Log-Linear para a instância [6.  2.7 5.1 1.6]: [('petal length (cm)', 5.1)]


c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


6.0: Comprimento da sépala (sepal length) em centímetros.
2.7: Largura da sépala (sepal width) em centímetros.
5.1: Comprimento da pétala (petal length) em centímetros.
1.6: Largura da pétala (petal width) em centímetros.

A PI-explicação sugere que o comprimento da pétala é o fator mais influente para a decisão do modelo sobre essa instância específica com 5.1 cm de comprimento.
Esse tipo de explicação ajuda a entender quais características são mais importantes para o modelo ao fazer uma previsão específica.

In [20]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carregando e preparando o dataset Iris com duas classes
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([1, 2])]  # Mudança para classes 1 e 2

# Separando os dados de treino e teste
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Regressão Logística
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade das duas classes previstas
    prob_predicted_class = clf.predict_proba(instance.reshape(1, -1))[0]

    # Calcula a diferença de probabilidade para cada atributo
    delta_features = []
    for i, feature in enumerate(features):
        coef = clf.coef_[0][i]  # Coeficiente do modelo para o atributo atual
        value = instance[i]     # Valor do atributo na instância atual
        delta = abs(coef * value)
        delta_features.append((feature, delta))


    # Ordena os atributos pela diferença de probabilidade (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = abs(prob_predicted_class[0] - prob_predicted_class[1])  # Diferença de probabilidade entre as duas classes
    current_threshold = 0
    for idx, (feature, delta) in enumerate(delta_features): 
        current_threshold += delta
        pi_explanation.append((feature, instance[features.index(feature)]))
        if current_threshold > threshold:
            break

    return pi_explanation

# Selecionando a primeira instância do conjunto de teste como um array NumPy
instance = X_test.iloc[0].to_numpy()

# Calculando a PI-explicação para a instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, list(X.columns))

# Exibindo a PI-explicação
print(f"PI-explicação Log-Linear para a instância {instance}: {pi_explanation}")


PI-explicação Log-Linear para a instância [6.3 2.8 5.1 1.5]: [('petal length (cm)', 5.1)]


c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [15]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from itertools import combinations

# Carregando e preparando o dataset Iris com duas classes
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([2, 0])]  # Mudança para classes 1 e 2

# Separando os dados de treino e teste
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Regressão Logística
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade das duas classes previstas
    prob_predicted_class = clf.predict_proba(instance.reshape(1, -1))[0]

    # Calcula a diferença de probabilidade para cada atributo
    delta_features = []
    for i, feature in enumerate(features):
        coef = clf.coef_[0][i]  # Coeficiente do modelo para o atributo atual
        value = instance[i]     # Valor do atributo na instância atual
        delta = abs(coef * value)
        delta_features.append((feature, delta))

    # Ordena os atributos pela diferença de probabilidade (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = abs(prob_predicted_class[0] - prob_predicted_class[1])  # Diferença de probabilidade entre as duas classes
    current_threshold = 0
    for idx, (feature, delta) in enumerate(delta_features): 
        current_threshold += delta
        pi_explanation.append((feature, instance[features.index(feature)]))
        if current_threshold > threshold:
            break

    return pi_explanation

def generate_all_minimal_explanations(instance, clf, features):
    """Gera todas as PI-explicações mínimas para uma instância.

    Args:
        instance: A instância para a qual se deseja calcular as PI-explicações (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de listas de pares valor-atributo, representando as PI-explicações mínimas.
    """

    minimal_explanations = []
    for i in range(1, len(features) + 1):
        for combination in combinations(features, i):
            explanation = [(feature, instance[features.index(feature)]) for feature in combination]
            if is_explanation_valid(instance, clf, explanation):
                minimal_explanations.append(explanation)
    return minimal_explanations

def is_explanation_valid(instance, clf, explanation):
    """Verifica se uma explicação é válida, ou seja, se ela contribui para a classe prevista.

    Args:
        instance: A instância para a qual se deseja verificar a explicação (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        explanation: A explicação a ser verificada (lista de pares valor-atributo).

    Returns:
        True se a explicação é válida, False caso contrário.
    """

    # Create a copy of X_train and fill in the values from the instance
    explanation_data = X_train.copy()
    for feature, value in explanation:
        explanation_data[feature] = value

    # Predict using the updated explanation_data
    predicted_class = clf.predict(explanation_data)[0]

    # Check if the predicted class matches the original class
    return predicted_class == clf.predict(instance.reshape(1, -1))[0]

def calculate_explanation_score(instance, clf, explanation, features):
    """Calcula o score de uma explicação.

    Args:
        instance: A instância para a qual se deseja calcular o score (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        explanation: A explicação para a qual se deseja calcular o score (lista de pares valor-atributo).
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        O score da explicação.
    """

    # Predict the class for the instance
    predicted_class = clf.predict(instance.reshape(1, -1))[0]

    # Get the probabilities for the instance
    probabilities = clf.predict_proba(instance.reshape(1, -1))[0]

    # Use the predicted class index to select the corresponding probability 
    prob_without_explanation = probabilities[predicted_class] 

    # Cria um novo conjunto de dados com apenas os atributos da explicação
    explanation_data = pd.DataFrame([instance], columns=X.columns)
    explanation_data = explanation_data[[feature for feature, _ in explanation]]

    # Calcula a probabilidade da classe prevista com a explicação
    prob_with_explanation = clf.predict_proba(explanation_data)[0][predicted_class]  # Corrected line

    # O score é a diferença entre as probabilidades
    score = prob_with_explanation - prob_without_explanation
    return score

def find_features_not_in_any_explanation(minimal_explanations, features):
    """Encontra os atributos que não estão em nenhuma das explicações mínimas.

    Args:
        minimal_explanations: Uma lista de listas de pares valor-atributo, representando as PI-explicações mínimas.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de atributos que não estão em nenhuma das explicações mínimas.
    """

    features_in_explanations = set()
    for explanation in minimal_explanations:
        for feature, _ in explanation:
            features_in_explanations.add(feature)

    features_not_in_explanations = [feature for feature in features if feature not in features_in_explanations]
    return features_not_in_explanations


# Selecionando a primeira instância do conjunto de teste como um array NumPy
instance = X_test.iloc[0].to_numpy()
features = list(X.columns)

# Calculando a PI-explicação para a instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, features)

# Gerando todas as explicações mínimas
minimal_explanations = generate_all_minimal_explanations(instance, clf, features)

# Calculando o score de cada explicação mínima
for explanation in minimal_explanations:
    score = calculate_explanation_score(instance, clf, explanation, features)
    print(f"Explicação: {explanation}, Score: {score}")

# Encontrando os atributos que não estão em nenhuma das explicações mínimas
features_not_in_explanations = find_features_not_in_any_explanation(minimal_explanations, features)
print(f"\nAtributos não presentes em nenhuma explicação: {features_not_in_explanations}")

c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarnin

IndexError: index 2 is out of bounds for axis 0 with size 2